In [ ]:
import requests
import gmaps as gm
import pandas as pd
import csv
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
#import census
#from census import Census

#initialize the census API
#c = Census("API_KEY")

# with open('apikey.rtf') as f:
#    api_key = f.readline()
#    f.close

# load the CSV file into a DataFrame
cities = pd.read_csv("Zip Code Params Benjamin Goroshnik.csv", usecols=['lat', 'lng', 'zip', 'city', 'state_id'], nrows=30000)#REMOVE
income = pd.read_csv("acs5.csv")

api_key = "API_KEY}"
radius = 10000  # 10km
types = "grocery store"
keywords = 'organic'

f = open('store_info_expanded4.csv', 'w', encoding='UTF8')
f.write("Name,Address,Lat,Lng,Phone,Rating,Costliness,Review Count,Chain Size,Status,Zip,Avg Income,Website\n")


# loop through all cities
for index, row in cities.iterrows():
    print("")
    city_name = row['city']
    state_id = row['state_id']
    zip = row['zip']
    print(f"City: {city_name}, State: {state_id}, zip: {zip}", end = "")
    
    latitude = row['lat']
    longitude = row['lng']
    first_call = True
    next_page_token = 'N/A'
    
    while (first_call or next_page_token != 'N/A'):
        # make a request to the places_nearby endpoint
        nearby_search_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}" \
              f"&radius={radius}&type={types}&keyword={keywords}&key={api_key}&nextPage()=true&hasNextPage=true"
        
        if (next_page_token != 'N/A'):
            nearby_search_url = nearby_search_url + f"&pagetoken={next_page_token}"
        
        #print (nearby_search_url)

        nearby_search_response = requests.get(nearby_search_url)
        nearby_search_data = nearby_search_response.json()

        next_page_token = nearby_search_data.get('next_page_token', 'N/A')
        
        if (first_call != True):
            print (f"next_page", end="")

        first_call = False


        # extract the location and contact information from the response
        for result in nearby_search_data["results"]:
            try:
                name = result["name"]
                #address = result["vicinity"]
                lat = result["geometry"]["location"]["lat"]
                lng = result["geometry"]["location"]["lng"]
                rating = result.get('rating', 'N/A')
                price_level = result.get('price_level', 'N/A')
                review_count = result.get('user_ratings_total', 'N/A')
                store_chain = result.get('chain', 'N/A')
                business_status = result.get('business_status')
                coord = f"{lat}, {lng}"
                location = geolocator.reverse(coord, timeout=10)
                address1 = location.raw['address']
                zip_code = address1.get('postcode', 'N/A')
                place_id = result["place_id"]

                #get place details
                place_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
                place_response = requests.get(place_url)
                place_data = place_response.json()
                place_result = place_data["result"]

                contact = place_result.get("international_phone_number","N/A")
                address = place_result.get("formatted_address","N/A")
                website = place_result.get("website","N/A")

                #find the record in income cvs by zip code, then take the first matching record and extract B19013_001E field from it
                try:
                    avg_income = income.loc[income.zip_code == int(zip_code)].iloc[0]['B19013_001E']
                except:
                    avg_income = "NA"
                #if zip_code != 'N/A':
                    #try:
                    #    income_data = c.acs5.zipcode(fields = ('NAME', 'B19013_001E'), zcta = "ZCTA5 " + zip_code, year = 2017)
                    #    avg_income = income_data[0]['B19013_001E']
                    #except:
                    #    avg_income = "NA"
                #print(f"Name: {name}, Address: {address}, Lat: {lat}, Lng: {lng}, Phone: {contact}, Rating: {rating}, Costliness: {price_level}, Review Count: {review_count}, Chain Size: {store_chain}, Status: {business_status}, Zip: {zip_code}, Avg Income: {avg_income}, website: {website}")
                #print(".", end ="")
                f.write(f"\"{name}\",\"{address}\",{lat},{lng},{contact},{rating},{price_level},{review_count},{store_chain},{business_status},{zip_code},{avg_income},{website}\n")
            except:
                print("skipping bad entry")
       
f.close()
print ("")
print ('Finished running!')


City: Seattle, State: WA, zip: 98116next_page
City: Chicago, State: IL, zip: 60615
City: Oakland, State: CA, zip: 94612next_page
City: Flushing, State: NY, zip: 11358next_page
City: Washington, State: DC, zip: 20007next_page
City: Berkeley, State: CA, zip: 94709
City: Los Angeles, State: CA, zip: 90035next_pagenext_page
City: Jamaica Plain, State: MA, zip: 2130next_page
City: Austin, State: TX, zip: 78701next_page
City: Bayside, State: NY, zip: 11360next_page
City: Jersey City, State: NJ, zip: 7304next_pagenext_page
City: Seattle, State: WA, zip: 98119next_page
City: Newark, State: NJ, zip: 7106
City: San Jose, State: CA, zip: 95113next_page
City: Redondo Beach, State: CA, zip: 90277
City: Oakland, State: CA, zip: 94618next_page
City: Dallas, State: TX, zip: 75219
City: Atlanta, State: GA, zip: 30326
City: Long Beach, State: CA, zip: 90814
City: Maspeth, State: NY, zip: 11378next_pagenext_page
City: Seattle, State: WA, zip: 98117next_page
City: Cambridge, State: MA, zip: 2138next_page